In [4]:
# imports
import os
from typing import List, Dict
from dotenv import load_dotenv
from openai import OpenAI
import ollama

# Constants
MODEL_GPT = "gpt-4o-mini"
MODEL_LLAMA = "llama3.2"

# Load environment variables
load_dotenv(override=True)

# Initialize OpenAI client
api_key = os.getenv("OPENAI_API_KEY")
if not api_key or not api_key.startswith("sk-") or len(api_key) < 10:
    raise ValueError("Invalid or missing OpenAI API key.")
openai_client = OpenAI()

# ----- Helper Functions -----
def build_messages(system_prompt: str, user_prompt: str) -> List[Dict[str, str]]:
    """Build the messages list for chat completion."""
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

# ----- Model Functions -----
def stream_gpt_response(system_prompt: str, user_prompt: str, model: str = MODEL_GPT) -> None:
    """Stream response from GPT model."""
    messages = build_messages(system_prompt, user_prompt)
    print("\n--- GPT Streaming Response ---\n")
    try:
        stream = openai_client.chat.completions.create(model=model, messages=messages, stream=True)
        for chunk in stream:
            if chunk.choices and chunk.choices[0].delta.get("content"):
                print(chunk.choices[0].delta.content, end="", flush=True)
        print("\n")
    except Exception as e:
        print(f"[Error from GPT model: {e}]")

def stream_llama_response(system_prompt: str, user_prompt: str, model: str = MODEL_LLAMA) -> None:
    """Stream response from LLaMA model."""
    messages = build_messages(system_prompt, user_prompt)
    print("\n--- LLaMA Streaming Response ---\n")
    try:
        stream = ollama.chat(model=model, messages=messages, stream=True)
        for chunk in stream:
            if "message" in chunk and "content" in chunk["message"]:
                print(chunk["message"]["content"], end="", flush=True)
        print("\n")
    except Exception as e:
        print(f"[Error from LLaMA model: {e}]")

# ----- Main -----
if __name__ == "__main__":
    # Static system prompt
    system_prompt = (
        "You are an expert Python tutor who explains code step-by-step, "
        "starting with a plain-English summary, then breaking down syntax, "
        "and finally explaining the reasoning behind using that code structure. "
        "Use clear examples and avoid jargon unless it is explained."
    )

    # Dynamic user prompt — just change the snippet to explain different code
    code_snippet = 'yield from {book.get("author") for book in books if book.get("author")}'
    user_prompt = f"Please explain what this Python code does and why:\n```python\n{code_snippet}\n```"

    # Stream responses
    stream_gpt_response(system_prompt, user_prompt)
    stream_llama_response(system_prompt, user_prompt)



--- LLaMA Streaming Response ---

**Plain-English Summary:**
This line of code generates a sequence (similar to a list or tuple) that contains the authors of all books in the `books` collection. The sequence is generated on-the-fly, rather than all at once, allowing for efficient processing of large datasets.

**Breaking Down Syntax:**

*   `{...}`: This is a dictionary comprehension syntax, which creates a new dictionary (in this case) based on the values obtained from the expression inside.
*   `book.get("author")`: This gets the value associated with the key `"author"` for each book in the `books` collection. If the key does not exist, it returns `None`.
*   `{...}`: The outer dictionary comprehension iterates over the books that have an author (i.e., those where `book.get("author") is not None`). For each such book, it adds its author to a new dictionary.
*   `yield from`: This keyword yields control back to the caller of the function, allowing the generator to produce values inst